In [15]:
import requests, json, uuid
#from docx import Document
import os
from dotenv import load_dotenv

load_dotenv()

subscription_key = os.getenv("TRANSLATOR_API_KEY")
endpoint = 'https://api.cognitive.microsofttranslator.com/'
location = os.getenv("TRANSLATOR_LOCATION") 
language_destination = 'pt-br'


def translator_text(text, language_destination):

  path = '/translate'
  constructed_url = 'https://api.cognitive.microsofttranslator.com/' + path

  params = {
      'api-version': '3.0',
      'from': 'en',
      'to': language_destination 
  }

  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(os.urandom(16))
  }


  body = [{
      'text': text
  }]


  response = requests.post(constructed_url, params=params, headers=headers, json=body)
  

  if response.status_code == 200:
    # call json() method on the response object
    response = response.json()

    if response and isinstance(response, list) and len(response) > 0 and \
       'translations' in response[0] and len(response[0]['translations']) > 0 and \
       'text' in response[0]['translations'][0]:
      return response[0]['translations'][0]['text']
    else:
      print("Error: Unexpected response format:", response)
      return None
  else:
      print("Error: API request failed with status code:", response.status_code)
      print("Error response:", response.text)
      return None

In [16]:
translator_text("Show me an open door", language_destination)


'Mostre-me uma porta aberta'

In [2]:
!pip install docx

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached lxml-5.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.8 kB)
Using cached lxml-5.3.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.9 MB)
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53894 sha256=f590336c55f19697b9495fe59094c345355697cb0f2cdb014cda104d24cd4a0d
  Stored in directory: /home/jcnok/.cache/pip/wheels/f3/ba/dd/43ed5f165600f41deddeb1e382c56ffc1067c09ec5bd705f39
Successfully built docx

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from src.utils.Config import Config
from src.services.blob_service import upload_to_blob

ModuleNotFoundError: No module named 'utils'

In [1]:
import os  


In [2]:
os.getcwd()

'/home/jcnok/bootcamps/Bootcamp-Microsoft-Certification-Challenge--1-AI_102/desafios_de_projeto/desafio_2/src'

In [3]:
from utils.Config import Config

In [4]:
from services.blob_service import upload_to_blob

In [ ]:
from azure.storage.blob import BlobServiceClient
from utils.Config import Config

def upload_to_blob(source, filename=None) -> str:
    """
    Faz o upload de uma imagem para o Azure Blob Storage

    Args:
        source: imagem com o caminho completo. ex:"img/imagem.png"
        filename:Nome opcional para o arquivo.

    Returns:
        str: URL do blob
    """
    if not filename:
        file = source.split(".")[0].split("/")[-1]
        extension = source.split(".")[-1]
        filename = f"{file}.{extension}"

    # Upload para o blob
    blob_service_client = BlobServiceClient.from_connection_string(
        Config.STORAGE_CONNECTION
    )
    blob_client = blob_service_client.get_blob_client(Config.CONTAINER_NAME, filename)
    with open(file=source, mode="rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    return blob_client.url